In [1]:
## temp file for GH's presentations


In [2]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'ContactToolAssociation__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"sf2021shitOPmlIiFMLnrudgC6oSX0WV1T",   
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [3]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI

In [4]:
db_1 = DB2SalesforceAPI(sf_login_params)
db_2 = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True
Obtained Salesforce access token ...... True


In [5]:
tool_cluster_df = db_1.query_data('Select ID, Ending_Date__c, Starting_Date__c, Organization__c,Estimated_cluster_size__c, \
                        Estimated_number_of_tools__c from tool_usage_cluster__c')

[Success] Bulk job creation successful. Job ID = 7505w00000TpyJcAAJ
{"id":"7505w00000TpyJcAAJ","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-02-09T17:04:09.000+0000","systemModstamp":"2021-02-09T17:04:09.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000TpyJcAAJ","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-02-09T17:04:09.000+0000","systemModstamp":"2021-02-09T17:04:17.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":3841,"retries":0,"totalProcessingTime":903}
[Success] Bulk job completed successfully.


In [6]:
display(tool_cluster_df.head(5))
print(tool_cluster_df.shape)

,Ending_Date__c,Estimated_cluster_size__c,Estimated_number_of_tools__c,Id,Organization__c,Starting_Date__c
0,2011-01-30,0.0,0.0,a0w5w000009Q7CuAAK,NaN,2010-09-05
1,2018-11-30,0.0,0.0,a0w5w000009Q7CvAAK,a0r5w00000V42iyAAB,2018-11-29
2,2019-11-28,0.0,0.0,a0w5w000009Q7CwAAK,a0r5w00000V42iyAAB,2019-11-04
3,2017-12-16,0.0,0.0,a0w5w000009Q7CxAAK,a0r5w00000V42cCAAR,2017-10-03
4,2018-01-21,0.0,0.0,a0w5w000009Q7CyAAK,a0r5w00000V42cCAAR,2018-01-15


(3841, 6)


In [7]:
## pull the list of contacts within each cluster   
contacts_in_cluster_df = db_2.query_data('Select Contact__c, Tool_Usage_Cluster__c from ContactToolClusterAssociation__c')

[Success] Bulk job creation successful. Job ID = 7505w00000TpyJhAAJ
{"id":"7505w00000TpyJhAAJ","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-02-09T17:04:21.000+0000","systemModstamp":"2021-02-09T17:04:21.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000TpyJhAAJ","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-02-09T17:04:21.000+0000","systemModstamp":"2021-02-09T17:04:22.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":1200,"retries":0,"totalProcessingTime":6343}
{"id":"7505w00000TpyJhAAJ","operation":"query","object":"ContactToolClusterAsso

In [10]:
display(contacts_in_cluster_df.head(5))
# contacts_in_cluster_df.to_csv('backup_contacts_in_clusters.csv')

,Contact__c,Tool_Usage_Cluster__c
0,0035w000034JlPmAAK,a0w5w00000A8UGwAAN
1,0035w000034JOZ4AAO,a0w5w000009Q8AEAA0
2,0035w000034JOZ4AAO,a0w5w00000A4L1HAAV
3,0035w000034K3MaAAK,a0w5w000009Q8SLAA0
4,0035w000034JJ7LAAW,a0w5w000009Q7tWAAS


In [11]:
contacts_in_cluster_df.shape

(62, 2)

In [13]:
if contacts_in_cluster_df.shape[0] < 100: #SF bugged once again
    contacts_in_cluster_df = pd.read_csv('backup_contacts_in_clusters.csv')
    print(contacts_in_cluster_df.shape)
    

In [14]:
contacts_in_cluster_df.shape

(92586, 3)

## determine the earliest cluster for an organization

In [16]:
# find all unique organizations
org_sorts = sorted(tool_cluster_df['Organization__c'].dropna().to_list())
print(org_sorts)

['a0r5w00000V42c0AAB', 'a0r5w00000V42c0AAB', 'a0r5w00000V42c0AAB', 'a0r5w00000V42c0AAB', 'a0r5w00000V42c0AAB', 'a0r5w00000V42c0AAB', 'a0r5w00000V42c0AAB', 'a0r5w00000V42c1AAB', 'a0r5w00000V42c1AAB', 'a0r5w00000V42c1AAB', 'a0r5w00000V42c1AAB', 'a0r5w00000V42c1AAB', 'a0r5w00000V42c1AAB', 'a0r5w00000V42c1AAB', 'a0r5w00000V42c1AAB', 'a0r5w00000V42c1AAB', 'a0r5w00000V42c1AAB', 'a0r5w00000V42c1AAB', 'a0r5w00000V42c1AAB', 'a0r5w00000V42c1AAB', 'a0r5w00000V42c1AAB', 'a0r5w00000V42c1AAB', 'a0r5w00000V42c3AAB', 'a0r5w00000V42c3AAB', 'a0r5w00000V42c3AAB', 'a0r5w00000V42c3AAB', 'a0r5w00000V42c4AAB', 'a0r5w00000V42cBAAR', 'a0r5w00000V42cBAAR', 'a0r5w00000V42cBAAR', 'a0r5w00000V42cBAAR', 'a0r5w00000V42cBAAR', 'a0r5w00000V42cCAAR', 'a0r5w00000V42cCAAR', 'a0r5w00000V42cCAAR', 'a0r5w00000V42cCAAR', 'a0r5w00000V42cCAAR', 'a0r5w00000V42cCAAR', 'a0r5w00000V42cCAAR', 'a0r5w00000V42cCAAR', 'a0r5w00000V42cCAAR', 'a0r5w00000V42cCAAR', 'a0r5w00000V42cCAAR', 'a0r5w00000V42cCAAR', 'a0r5w00000V42cCAAR', 'a0r5w000

In [27]:
import itertools 
org_sorts_grouped = itertools.groupby(org_sorts)

In [28]:
unique_orgs = []
for i,j in enumerate(org_sorts_grouped):
    unique_orgs.append(j[0])

In [31]:
import numpy as np
import datetime

In [69]:
# for each organization, determine the earliest date
earliest_year = []
full_orgs_sf = tool_cluster_df['Organization__c'].to_list()
for i,j in enumerate(unique_orgs):
    # determine all row indexes in tool_cluster_df that contain said organization
    temp = np.in1d(full_orgs_sf,j)
    row_inds = np.where(temp == True)
    
    dt_format = pd.DatetimeIndex(pd.to_datetime(tool_cluster_df['Starting_Date__c']).to_numpy()[row_inds])
    earliest_year.append(dt_format[np.argmin(dt_format)].year)

In [71]:
## populate df's column with said entry
sf_new_df = pd.DataFrame()
sf_new_df['Organization__c'] = unique_orgs
sf_new_df['YearofF__c'] = earliest_year
display(sf_new_df.head(5))
display(sf_new_df.tail(5))

,Organization__c,YearofF__c
0,a0r5w00000V42c0AAB,2006
1,a0r5w00000V42c1AAB,2010
2,a0r5w00000V42c3AAB,2014
3,a0r5w00000V42c4AAB,2020
4,a0r5w00000V42cBAAR,2008


,Organization__c,YearofF__c
324,a0r5w00000V4H98AAF,2019
325,a0r5w00000V4H99AAF,2011
326,a0r5w00000V4HAeAAN,2017
327,a0r5w00000V4HBuAAN,2005
328,a0r5w00000VNgG2AAL,2012


In [93]:
## populate total clusters per org
org_sorts_grouped = itertools.groupby(org_sorts)

In [94]:
counter = 0
total_clusters = []
for i,j in enumerate(org_sorts_grouped):
    total_clusters.append(sum(1 for x in j[1]))
    

In [95]:
len(total_clusters)

329

In [96]:
sf_new_df['Total_Clusters__c'] = total_clusters
display(sf_new_df.head(5))
display(sf_new_df.tail(5))

,Organization__c,YearofF__c,Total_Clusters__c
0,a0r5w00000V42c0AAB,2006,7
1,a0r5w00000V42c1AAB,2010,15
2,a0r5w00000V42c3AAB,2014,4
3,a0r5w00000V42c4AAB,2020,1
4,a0r5w00000V42cBAAR,2008,5


,Organization__c,YearofF__c,Total_Clusters__c
324,a0r5w00000V4H98AAF,2019,1
325,a0r5w00000V4H99AAF,2011,7
326,a0r5w00000V4HAeAAN,2017,2
327,a0r5w00000V4HBuAAN,2005,12
328,a0r5w00000VNgG2AAL,2012,1


In [136]:
## develop Clusters_Yr2020 and Clusters_Yr2019
Clusters_Yr2020 = []
Clusters_Yr2019 = []
start_dates = pd.to_datetime(tool_cluster_df['Starting_Date__c'])
end_dates = pd.to_datetime(tool_cluster_df['Ending_Date__c'])

orgs_2020 = tool_cluster_df['Organization__c'][(start_dates >= '2020') & (start_dates < '2021')].reset_index()['Organization__c']
orgs_2019 = tool_cluster_df['Organization__c'][(start_dates >= '2019') & (start_dates < '2020')].reset_index()['Organization__c']

for i,j in enumerate(unique_orgs):
    # find all relevant clusters
    Clusters_Yr2020.append(sum([1 for i in np.where(np.in1d(orgs_2020,j) == True)[0]]))
    Clusters_Yr2019.append(sum([1 for i in np.where(np.in1d(orgs_2019,j) == True)[0]]))

In [141]:
sf_new_df['Clusters_Yr2020__c'] = Clusters_Yr2020
sf_new_df['Clusters_Yr2019__c'] = Clusters_Yr2019

In [144]:
display(sf_new_df.head(5))
display(sf_new_df.tail(5))

,Organization__c,YearofF__c,Total_Clusters__c,Clusters_Yr2020__c,Clusters_Yr2019__c
0,a0r5w00000V42c0AAB,2006,7,1,0
1,a0r5w00000V42c1AAB,2010,15,4,2
2,a0r5w00000V42c3AAB,2014,4,0,0
3,a0r5w00000V42c4AAB,2020,1,1,0
4,a0r5w00000V42cBAAR,2008,5,0,1


,Organization__c,YearofF__c,Total_Clusters__c,Clusters_Yr2020__c,Clusters_Yr2019__c
324,a0r5w00000V4H98AAF,2019,1,0,1
325,a0r5w00000V4H99AAF,2011,7,0,1
326,a0r5w00000V4HAeAAN,2017,2,0,1
327,a0r5w00000V4HBuAAN,2005,12,0,0
328,a0r5w00000VNgG2AAL,2012,1,0,0


In [ ]:
## building ClusteredUsers_2020__c and ClusteredUsers_2019__c

In [156]:
## develop Clusters_Yr2020 and Clusters_Yr2019
ClusteredUsers_2020__c = []
ClusteredUsers_2019__c = []
start_dates = pd.to_datetime(tool_cluster_df['Starting_Date__c'])
end_dates = pd.to_datetime(tool_cluster_df['Ending_Date__c'])

orgs_2020 = tool_cluster_df['Organization__c'][(start_dates >= '2020') & (start_dates < '2021')].reset_index()['Organization__c']
orgs_2019 = tool_cluster_df['Organization__c'][(start_dates >= '2019') & (start_dates < '2020')].reset_index()['Organization__c']

ids_2020 = tool_cluster_df['Id'][(start_dates >= '2020') & (start_dates < '2021')].reset_index()['Id']
ids_2019 = tool_cluster_df['Id'][(start_dates >= '2019') & (start_dates < '2020')].reset_index()['Id']

contacts_in_clusters = contacts_in_cluster_df['Tool_Usage_Cluster__c']

for i,j in enumerate(unique_orgs):
    # find all relevant clusters
    temp_2020 = ids_2020[np.where(np.in1d(orgs_2020,j) == True)[0]]
    temp_2019 = ids_2019[np.where(np.in1d(orgs_2019,j) == True)[0]]
    
    #print(np.where(np.in1d(contacts_in_clusters,temp_2020) == True))
    #print(np.where(np.in1d(contacts_in_clusters,temp_2019) == True))
    
    ClusteredUsers_2020__c.append(sum([1 for i in np.where(np.in1d(contacts_in_clusters,temp_2020) == True)[0]]))
    ClusteredUsers_2019__c.append(sum([1 for i in np.where(np.in1d(contacts_in_clusters,temp_2019) == True)[0]]))
    

In [159]:
sf_new_df['ClusteredUsers_2020__c'] = ClusteredUsers_2020__c
sf_new_df['ClusteredUsers_2019__c'] = ClusteredUsers_2019__c
display(sf_new_df.head(5))
display(sf_new_df.tail(5))

,Organization__c,YearofF__c,Total_Clusters__c,Clusters_Yr2020__c,Clusters_Yr2019__c,ClusteredUsers_2020__c,ClusteredUsers_2019__c
0,a0r5w00000V42c0AAB,2006,7,1,0,5,0
1,a0r5w00000V42c1AAB,2010,15,4,2,54,32
2,a0r5w00000V42c3AAB,2014,4,0,0,0,0
3,a0r5w00000V42c4AAB,2020,1,1,0,11,0
4,a0r5w00000V42cBAAR,2008,5,0,1,0,9


,Organization__c,YearofF__c,Total_Clusters__c,Clusters_Yr2020__c,Clusters_Yr2019__c,ClusteredUsers_2020__c,ClusteredUsers_2019__c
324,a0r5w00000V4H98AAF,2019,1,0,1,0,10
325,a0r5w00000V4H99AAF,2011,7,0,1,0,24
326,a0r5w00000V4HAeAAN,2017,2,0,1,0,9
327,a0r5w00000V4HBuAAN,2005,12,0,0,0,0
328,a0r5w00000VNgG2AAL,2012,1,0,0,0,0


In [162]:
sf_new_df = sf_new_df.rename(columns={'Organization__c':'Id'})

In [163]:
sf_new_df

,Id,YearofF__c,Total_Clusters__c,Clusters_Yr2020__c,Clusters_Yr2019__c,ClusteredUsers_2020__c,ClusteredUsers_2019__c
0,a0r5w00000V42c0AAB,2006,7,1,0,5,0
1,a0r5w00000V42c1AAB,2010,15,4,2,54,32
2,a0r5w00000V42c3AAB,2014,4,0,0,0,0
3,a0r5w00000V42c4AAB,2020,1,1,0,11,0
4,a0r5w00000V42cBAAR,2008,5,0,1,0,9
...,...,...,...,...,...,...,...
324,a0r5w00000V4H98AAF,2019,1,0,1,0,10
325,a0r5w00000V4H99AAF,2011,7,0,1,0,24
326,a0r5w00000V4HAeAAN,2017,2,0,1,0,9
327,a0r5w00000V4HBuAAN,2005,12,0,0,0,0


In [164]:
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = 'Organization__c'
db_s.external_id = 'Id'

db_s.send_data(sf_new_df)

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000TpzIkAAJ
hello
[Success] CSV upload successful. Job ID = 7505w00000TpzIkAAJ
[Success] Closing job successful. Job ID = 7505w00000TpzIkAAJ


In [165]:
db_s.check_bulk_status()

{'id': '7505w00000TpzIkAAJ',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-02-09T18:52:15.000+0000',
 'systemModstamp': '2021-02-09T18:52:20.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 329,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 1184,
 'apiActiveProcessingTime': 957,
 'apexProcessingTime': 0}